In [ ]:
import feedparser 
from urllib.parse import urlparse, parse_qs, quote
import requests
from bs4 import BeautifulSoup
import json

In [9]:
def resolve_google_news_url(google_url):
    resp = requests.get(google_url)
    data = BeautifulSoup(resp.text, 'html.parser').select_one('c-wiz[data-p]').get('data-p')
    obj = json.loads(data.replace('%.@.', '["garturlreq",'))

    payload = {
        'f.req': json.dumps([[['Fbv4je', json.dumps(obj[:-6] + obj[-2:]), 'null', 'generic']]])
    }

    headers = {
    'content-type': 'application/x-www-form-urlencoded;charset=UTF-8',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36',
    }

    url = "https://news.google.com/_/DotsSplashUi/data/batchexecute"
    response = requests.post(url, headers=headers, data=payload)
    array_string = json.loads(response.text.replace(")]}'", ""))[0][2]
    article_url = json.loads(array_string)[1]
    return article_url

In [ ]:
def fetch_article_content(url):
    """
    Fetch article text from the resolved publisher URL.
    """
    try:
        response = requests.get(url, timeout=10,
                                headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Grab all paragraph text
        paragraphs = soup.find_all('p')
        content = ' '.join([p.get_text() for p in paragraphs])
        # Remove common boilerplate phrases
        boilerplate_phrases = [
            "Oops, something went wrong",
            "Something went wrong"
        ]
        for phrase in boilerplate_phrases:
            content = content.replace(phrase, "")
        return content.strip() if content else "No readable content found."
    except requests.RequestException:
        return "Content not retrieved."


In [11]:
def fetch_news(query, num_articles=5):
    rss_url = f"https://news.google.com/rss/search?q={quote(query)}"
    feed = feedparser.parse(rss_url)
    news_items = feed.entries[:num_articles]

    articles = []
    for item in news_items:
        title = item.title
        link = resolve_google_news_url(item.link)
        published = item.published
        content = fetch_article_content(link)
        articles.append({
            "title": title,
            "link": link,
            "published": published,
            "content": content
        })

    return articles

In [17]:
fetch_news("Apple Inc", 1)

[{'title': "Apple's Upcoming Foldable iPhone Model Will Drive Major Upside: Analyst - Yahoo Finance",
  'link': 'https://finance.yahoo.com/news/apples-upcoming-foldable-iphone-model-023110451.html',
  'published': 'Wed, 29 Oct 2025 02:31:00 GMT',
  'content': 'Something went wrong Benzinga and Yahoo Finance LLC may earn commission or revenue on some items through the links below. Growing enthusiasm around Apple Inc‘s (NASDAQ:AAPL) iPhone 17 demand and anticipation for next year\'s foldable iPhone 18 has analysts optimistic about the company\'s multi-year growth outlook. JPMorgan analyst Samik Chatterjee, for example, says the Cupertino, California-based company heads into its upcoming earnings report with its strongest investor sentiment in a year. Trending: Missed Nvidia and Tesla? RAD Intel Could Be the Next AI Powerhouse — Invest Now at Just $0.81 a Share Improved data on iPhone demand and stabilization across Apple\'s broader business have helped rebuild investor confidence, Chatte